## Домашнее задание №2

### Часть 1
Нужно подобрать оптимальные параметры xgboost для набора данных <b>HR.csv</b> в папке второго семинара. Код для проверки качества представлен в скрипте <b>xgboost_params_checker.py</b>, а пример набора параметров в <b>xgboost_params_example.json</b>. Чекер с вашими параметрами должен отработать за 2 минуты на машинке для проверки. Для сравнения на xgboost_params_example.json чекер работает 20 секунд.

In [2]:
from sklearn.model_selection import cross_val_score
import os
import json
import sys
import signal


import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

import dask_searchcv as dcv
from dask.diagnostics import ProgressBar



PATH_TO_DATA="/Users/roman/DMIA/industry/seminar02/HR.csv"

/Users/roman/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


_____

In [3]:
data=pd.read_csv(PATH_TO_DATA)

X,y=data.drop("left",axis=1),data["left"]

In [4]:
xgb=XGBClassifier(seed=42,nthread=-1)


param_grid={"learning_rate": np.linspace(0.0008,0.001,20), 
            "max_depth": range(6,10), 
            "n_estimators": [3700], 
            "min_child_weight": [1,2,3,4,5],
           }

In [ ]:
#Считалось на google gloud

#dcv.GridSearchCV в связке с with ProgressBar():search.fit(X,y) позволяют видеть прогресс бар

search = dcv.GridSearchCV(xgb, param_grid,cv=3,n_jobs=-1,scoring='accuracy')

with ProgressBar():
    search.fit(X,y)

In [6]:
#search.best_params_
{
    "learning_rate": 0.00088105263157894732, 
    "max_depth": 8, 
    "n_estimators": 3200, 
    "min_child_weight": 1,
    "seed": 42
}
#чутка подкрутил руками чтоб в 1 мин укладывалось

{'learning_rate': 0.0008810526315789474,
 'max_depth': 8,
 'min_child_weight': 1,
 'n_estimators': 3200,
 'seed': 42}

### Проверка занания 1

In [8]:
%%time
SCRIPT_DIR = os.path.dirname(os.path.realpath("xgboost_params_checker.py"))


def signal_handler(signum, frame):
    raise Exception("Timed out!")


class Checker(object):
    def __init__(self, data_path=SCRIPT_DIR + '/../seminar02/HR.csv'):
        df = pd.read_csv(data_path)
        target = 'left'
        features = [c for c in df if c != target]
        self.target = np.array(df[target])
        self.data = np.array(df[features])

    def check(self, params_path):
        try:
            with open(params_path, 'r') as f:
                params = json.load(f)
                signal.signal(signal.SIGALRM, signal_handler)
                # Time limit на эту задачу 2 минуты
                signal.alarm(120)
                estimator = XGBClassifier(**params)
                score = np.mean(cross_val_score(
                    estimator, self.data, self.target,
                    scoring='accuracy', 
                    cv=3
                ))
        except:
            print("Unexpected error:", sys.exc_info()[0])
            score = None
        
        return score


if __name__ == '__main__':
    print(Checker().check("xgboost_matiiv.json"))
#     print(Checker().check("xgboost_params_example.json"))


0.782188224312
CPU times: user 1min 1s, sys: 160 ms, total: 1min 1s
Wall time: 1min 1s


### Часть 2

Нужно реализовать градиентный бустинг для задачи классификации на наборе данных <b>HR.csv</b>. Код для проверки качества представлен в скрипте <b>gb_impl_checker.py</b>, а шаблон, от которого стоит отталкиваться, в скрипте <b>gb_impl_example.py</b>. Чекер вашего скрипта должен отработать за 1 минуту на машинке для проверки. Для сравнения на <b>gb_impl_example.py</b> чекер работает 10 секунд.

Задание сдавать в форму https://goo.gl/forms/cfzYQ3jVnNjG2k1B3

Так как нужно присылать файлы, а коды чекеров заранее выложены, то у вас будет только одна посылка.

In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
import itertools
from tqdm import tqdm
PATH_TO_DATA="/Users/roman/DMIA/industry/seminar02/HR.csv"

data=pd.read_csv(PATH_TO_DATA)
X,y=data.drop("left",axis=1),data["left"]

In [2]:
from sklearn.base import BaseEstimator
from sklearn.tree import DecisionTreeRegressor
import numpy as np


# Параметрами с которыми вы хотите обучать деревья
TREE_PARAMS_DICT = {'max_depth': 1}
# Параметр tau (learning_rate) для вашего GB
TAU = 0.05


class SimpleGB(BaseEstimator):
    def __init__(self, tree_params_dict, iters, tau):
        self.tree_params_dict = tree_params_dict
        self.iters = iters
        self.tau = tau
        
    def fit(self, X_data, y_data):
        self.base_algo = DecisionTreeRegressor(**self.tree_params_dict,random_state=42).fit(X_data, y_data)
        self.estimators = []
        curr_pred = self.base_algo.predict(X_data)
        for iter_num in range(self.iters):
            # Нужно посчитать градиент функции потерь
            grad=y_data/(1+np.exp(-y_data*curr_pred)) # TODO
            # Нужно обучить DecisionTreeRegressor предсказывать антиградиент
            # Не забудьте про self.tree_params_dict
            algo = DecisionTreeRegressor(**self.tree_params_dict,random_state=42).fit(X_data, -grad) # TODO

            self.estimators.append(algo)
            # Обновите предсказания в каждой точке
            curr_pred +=self.tau*algo.predict(X_data)  # TODO
        return self
    
    def predict(self, X_data):
        # Предсказание на данных
        res = self.base_algo.predict(X_data)
        for estimator in self.estimators:
            res += self.tau * estimator.predict(X_data)
        # Задача классификации, поэтому надо отдавать 0 и 1
        return res > 0.

In [4]:
%%time
# Проверка
algo = SimpleGB(tree_params_dict=TREE_PARAMS_DICT,iters=100,tau=TAU)
print(np.mean(cross_val_score(algo, X, y, cv=3, scoring='accuracy')))

0.654774088151
CPU times: user 1.06 s, sys: 55.8 ms, total: 1.11 s
Wall time: 1.17 s


#### Подбор параметров для SimpleGB

In [10]:
TREE_PARAMS_DICT={"max_depth":[1,5,9],
                  "min_samples_split":range(2,9),
                  "min_samples_leaf":range(2,9),
                  "max_features":[1,7,None],
                  "TAU":np.linspace(0.5,1,5)
                }

In [11]:
#Долго
res={}
for i in tqdm(itertools.product(TREE_PARAMS_DICT["max_depth"],
                                TREE_PARAMS_DICT["min_samples_split"],
                                TREE_PARAMS_DICT["min_samples_leaf"],
                                TREE_PARAMS_DICT["max_features"],
                                TREE_PARAMS_DICT["TAU"])):
    tmp={"max_depth":i[0],
        "min_samples_split":i[1],
        "min_samples_leaf":i[2],
        "max_features":i[3]}
    algo = SimpleGB(tree_params_dict=tmp,iters=100,tau=i[4])
#     accuracy=np.mean(cross_val_score(algo, X, y, cv=3, scoring='accuracy'))
#     res[accuracy]=i

2205it [00:00, 317586.63it/s]


In [128]:
res[max(res)]

{0.65477408815096361: (9, 10, 10, None, 0.8666666666666667),
 0.65484075481763027: (9, 10, 5, None, 0.8666666666666667)}

### Проверка 2 задания

In [2]:
%%time
from sklearn.model_selection import cross_val_score
import numpy as np
import os
import sys
import imp
import signal
import pandas as pd


SCRIPT_DIR = os.path.dirname(os.path.realpath("gb_impl_checker.py"))


def signal_handler(signum, frame):
    raise Exception("Timed out!")


class Checker(object):
    def __init__(self, data_path=SCRIPT_DIR + '/../seminar02/HR.csv'):
        df = pd.read_csv(data_path)
        target = 'left'
        features = [c for c in df if c != target]
        self.target = np.array(df[target])
        self.data = np.array(df[features])
        self.application = 0

    def check(self, script_path):
        try:
            signal.signal(signal.SIGALRM, signal_handler)
            # Time limit на эту задачу 1 минута
            signal.alarm(60)
            gb_impl = imp.load_source('gb_impl_{}'.format(self.application), script_path)
            self.application += 1
            # Обучаться будет на 100 итерациях, чтобы одинаковый масштаб был
            algo = gb_impl.SimpleGB(
                tree_params_dict=gb_impl.TREE_PARAMS_DICT,
                iters=100,
                tau=gb_impl.TAU
            )
            return np.mean(cross_val_score(algo, self.data, self.target, cv=3, scoring='accuracy'))
        except:
            print("Unexpected error:", sys.exc_info()[0])
            return None


if __name__ == '__main__':
    print(Checker().check(SCRIPT_DIR + '/mygb_matiiv.py'))
#     print(Checker().check(SCRIPT_DIR + '/gb_impl_example.py'))

0.654774088151
CPU times: user 2.77 s, sys: 10.9 ms, total: 2.78 s
Wall time: 2.79 s
